# 라이브러리 및 데이터

In [0]:
# base
import os
import shutil
import random
import numpy as np
import time
from collections import Counter
from sklearn.metrics import mean_squared_error

# data preprocessing
import pandas as pd
pd.set_option('max_columns', 130, 'max_rows', 30)

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

# ignore warining
import warnings
warnings.filterwarnings('ignore')


import folium # 지도 관련 시각화
from folium.plugins import MarkerCluster #지도 관련 시각화
import geopy.distance #거리 계산해주는 패키지 사용


# save
from sklearn.externals import joblib 
import pickle

# selenium
from selenium.webdriver import Chrome

import geopy.distance

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals import joblib 
import pickle

import random
random.seed(42)

import lightgbm as lgb
from keras import metrics
from sklearn.metrics import r2_score

In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
bts = pd.read_csv("bus_bts.csv")

train.shape, test.shape, bts.shape

CRAWLING CODE
기상청 날씨누리 사이트에서 지상 관측 자료를 크롤링 함
http://www.weather.go.kr/
중급자 코드가 올라오기 전에 수행 함
중급자 코드에서 weather.csv를 만드는 코드가 있는데 제주감귤팀이 제출한 weather.csv는 이 크롤링을 통해 만들어진 것이고, 중급자 코드에 해당하는 weather.csv가 아님
chromedriver.exe를 작업 경로에 위치시켜야 함
2019년 9월 1일부터 2019년 10월 16일 까지의 데이터를 크롤링하였으며, 오전 10시의 관측 데이터만 사용하였기 때문에 data leakage에 해당하지 않는다.

In [0]:
def crawl_weather() :
    
    weather_data_10 = pd.DataFrame(columns=['현재일기_10','현재기온_10','체감온도_10','일강수_10'])
    browser = Chrome()
    url = 'http://www.weather.go.kr/weather/observation/currentweather.jsp?auto_man=m&type=t99&reg=184&tm=2019.10.25.16%3A00&x=19&y=7'
    browser.get(url)

    for i in range(0,46):
        i+=1

        elem=browser.find_element_by_id('observation_text')
        elem.clear()
        elem.send_keys("2019.9.{}.10:00".format(i))

        btn=browser.find_elements_by_class_name('btn')
        btn[2].click()
        
        time.sleep(1)
        weathers = browser.find_elements_by_css_selector('td')
        weather_data_10 = weather_data_10.append(pd.DataFrame([[weathers[40].text,weathers[44].text, weathers[46].text, weathers[47].text]],columns=['현재일기_10','현재기온_10','체감온도_10','일강수_10']))
        
            
    print('success !')
    browser.close()
    
    return weather_data_10

In [0]:
weather_data = crawl_weather()
weather_data.to_csv('weather.csv', index = False)

print('save.. !')
weather_data = pd.read_csv('weather.csv')

# 데이터 전처리
#탐색적 자료분석
#변수 선택 및 모델 구축


In [0]:
# weekday
train['date'] = pd.to_datetime(train['date'])
train['weekday'] = train['date'].dt.weekday

test['date'] = pd.to_datetime(test['date'])
test['weekday'] = test['date'].dt.weekday

train.shape, test.shape

In [0]:
# route_station = bus_route_id + station_code
train['bus_route_id'] = train['bus_route_id'].astype(str)
train['station_code'] = train['station_code'].astype(str)
train['route_station'] = train['bus_route_id'] + ',' + train['station_code']

test['bus_route_id'] = test['bus_route_id'].astype(str)
test['station_code'] = test['station_code'].astype(str)
test['route_station'] = test['bus_route_id'] + ',' + test['station_code']

train.shape, test.shape

In [0]:
# bus_route_id_weekday = bus_route_id + weekday
train['bus_route_id_weekday'] = train['bus_route_id'].astype(str) + ',' + train['weekday'].astype(str) 
test['bus_route_id_weekday'] = test['bus_route_id'].astype(str) + ',' + test['weekday'].astype(str) 

train.shape, test.shape

In [0]:
# station_code_weekday = station_code + weekday
train['station_code_weekday'] = train['station_code'].astype(str) + ',' + train['weekday'].astype(str)
test['station_code_weekday'] = test['station_code'].astype(str) + ',' + test['weekday'].astype(str)

train.shape, test.shape

In [0]:
# route_station_weekday = route_station + weekay
train['route_station_weekday'] = train['route_station'].astype(str) + ',' + train['weekday'].astype(str) 
test['route_station_weekday'] = test['route_station'].astype(str) + ',' + test['weekday'].astype(str)

train.shape, test.shape

In [0]:
# on_time - bts.csv 데이터에서 geton_time 열에서 시간대만 추출하여 on_time 컬럼을 만듬
bts['on_time']  = bts['geton_time'].apply(lambda x : x[:2])

bts.iloc[bts.query('on_time == "06"').index,13] = '6~7_ride'
bts.iloc[bts.query('on_time == "07"').index,13] = '7~8_ride'
bts.iloc[bts.query('on_time == "08"').index,13] = '8~9_ride'
bts.iloc[bts.query('on_time == "09"').index,13] = '9~10_ride'
bts.iloc[bts.query('on_time == "10"').index,13] = '10~11_ride'
bts.iloc[bts.query('on_time == "11"').index,13] = '11~12_ride'

bts.shape

In [0]:
# 승 하차 시간대 통합 변수 (t ~ t+2)
# t~t+1, t+1~t+2 시간대 승하차인원을 합하여 t~t+2 시간대 승하차인원 변수를 만듬
train['68a']=train['6~7_ride']+train['7~8_ride'] 
train['810a']=train['8~9_ride']+train['9~10_ride']
train['1012a']=train['10~11_ride']+train['11~12_ride']

train['68b']=train['6~7_takeoff']+train['7~8_takeoff'] 
train['810b']=train['8~9_takeoff']+train['9~10_takeoff']
train['1012b']=train['10~11_takeoff']+train['11~12_takeoff']

test['68a']=test['6~7_ride']+test['7~8_ride']
test['810a']=test['8~9_ride']+test['9~10_ride']
test['1012a']=test['10~11_ride']+test['11~12_ride']

test['68b']=test['6~7_takeoff']+test['7~8_takeoff']
test['810b']=test['8~9_takeoff']+test['9~10_takeoff']
test['1012b']=test['10~11_takeoff']+test['11~12_takeoff']

train.shape, test.shape

In [0]:
# Make features by using target variable
# 우리가 최종적으로 예측해야할 것은 각 일자별(date), 버스 노선(bus_route_id) 상의 정류장(station_name)의 퇴근시간 하차인원(18~20_ride)이다.

# bus_route_id, station_name, weekday의 각 조합별 퇴근시간 하차인원(18~20_ride)의 여러 통계량을 구한 후 이를 train set, test set에 모두 적용한다."
# target 변수를 train, test set에 적용할 수 있는 이유는 우리가 예측해야할 id는 date, bus_rout_id, station_name으로 구성되어있기 때문이다. 즉, 각각의 노선, 정류장별로 공통적인 패턴이 존재할 수 있다.
# 이 과정에서 NA 값이 생기는 이유는 train set에 없는 bus_route_id, station_name이 존재하기 때문이다.
def id_statistic(ID, col1, col2) :
    
    # mean, sum
    rs_mean = train.groupby([ID])['18~20_ride'].agg([(col1, 'mean')]).reset_index()
    rs_sum = train.groupby([ID])['18~20_ride'].agg([(col2, 'sum')]).reset_index()
    rs_mean_sum = pd.merge(rs_mean, rs_sum, on=ID)

    # merge
    tr = pd.merge(train, rs_mean_sum, how='left', on=ID)
    te = pd.merge(test, rs_mean_sum, how='left', on=ID)

    # na -&gt; mean
    te[col1] = te[col1].fillna(rs_mean.mean())
    te[col1] = te[col1].fillna(rs_sum.mean())
    
    return tr, te

In [0]:
train, test = id_statistic('route_station', '1820_rs_mean', '1820_rs_sum')

train.shape, test.shape

train, test = id_statistic('bus_route_id', '1820_r_mean', '1820_r_sum')

train.shape, test.shape

train, test = id_statistic('station_code', '1820_s_mean', '1820_s_sum')

train.shape, test.shape

train, test = id_statistic('weekday', '1820_w_mean', '1820_w_sum')

train.shape, test.shape

In [0]:
def mean_statistics() :

    f = train.groupby(['bus_route_id_weekday'])['18~20_ride'].agg([('mean_bus_weekday_ride','mean')]).reset_index()
    tr = pd.merge(train, f, how='left', on='bus_route_id_weekday')
    te = pd.merge(test, f, how='left', on='bus_route_id_weekday').fillna(f['mean_bus_weekday_ride'].mean())
    
    f = train.groupby(['station_code_weekday'])['18~20_ride'].agg([('mean_station_weekday_ride','mean')]).reset_index()
    tr = pd.merge(tr, f, how='left', on='station_code_weekday')
    te = pd.merge(te, f, how='left', on='station_code_weekday').fillna(f['mean_station_weekday_ride'].mean())
    
    f = train.groupby(['route_station_weekday'])['18~20_ride'].agg([('mean_route_station_weekday_ride','mean')]).reset_index()
    tr = pd.merge(tr, f, how='left', on='route_station_weekday')
    te = pd.merge(te, f, how='left', on='route_station_weekday').fillna(f['mean_route_station_weekday_ride'].mean())
    
    return tr, te

In [0]:
train, test = mean_statistics()

train.shape, test.shape

In [0]:
# congestion - bus_route_id를 기준으로 18 ~ 20의 혼잡도를 계산한다.
def congestion() :
    df = train.groupby(['bus_route_id'])['18~20_ride'].agg([('passenger', 'sum')])
    df = df.sort_values(by='passenger', ascending=False).reset_index()
    
    def f(x):
        if x &gt; 10000:
            return 7

        elif x &gt; 5000:
            return 6

        elif x &gt; 2000:
            return 5

        elif x &gt; 700:
            return 4

        elif x &gt; 200:
            return 3

        elif x &gt; 50:
            return 2

        else:
            return 1
    
    df['congestion']=df['passenger'].apply(f)
    df = df[['bus_route_id','congestion']]
    
    tr = pd.merge(train, df, how='left', on='bus_route_id')
    te = pd.merge(test, df, how='left', on='bus_route_id')
    
    # 결측치는 데이터 프레임 df의 'congestion'의 중간값인 '4'으로 대체
    te = te.fillna(4)
    
    return tr, te

In [0]:
train, test = congestion()

train.shape, test.shape

In [0]:
# location = latitude + longitude
train['location'] = train['latitude'].astype(str) + ',' + train['longitude'].astype(str)
test['location'] = test['latitude'].astype(str) + ',' + test['longitude'].astype(str)

train.shape, test.shape

In [0]:
# merge key
# make cue column
train['cue']=0
test['cue']=1

train.shape, test.shape

In [0]:
# 오전 시간의 여러 데이터 활용한 변수
# 오전 시간의 탑승 및 하차 데이터를 활용하여 요약통계량을 만든다.
def morning() :
    
    # merge
    data = pd.concat([train, test])
    
    a = data.groupby(['route_station'])['1012a'].agg({'sum', 'mean'}).reset_index()
    a.columns = ['route_station', '1012a_sum','1012a_mean']

    b = data.groupby(['route_station'])['1012b'].agg({'sum', 'mean'}).reset_index()
    b.columns = ['route_station', '1012b_sum','1012b_mean']
    b = b[['1012b_sum','1012b_mean']]

    c = data.groupby(['route_station'])['10~11_ride'].agg({'sum', 'mean'}).reset_index()
    c.columns = ['route_station', '10~11_ride_sum','10~11_ride_mean']
    c = c[['10~11_ride_sum','10~11_ride_mean']]

    d = data.groupby(['route_station'])['10~11_takeoff'].agg({'sum', 'mean'}).reset_index()
    d.columns = ['route_station', '10~11_takeoff_sum','10~11_takeoff_mean']
    d = d[['10~11_takeoff_sum','10~11_takeoff_mean']]

    e = data.groupby(['route_station'])['11~12_ride'].agg({'sum', 'mean'}).reset_index()
    e.columns = ['route_station', '11~12_ride_sum','11~12_ride_mean']
    e = e[['11~12_ride_sum','11~12_ride_mean']]

    f = data.groupby(['route_station'])['11~12_takeoff'].agg({'sum', 'mean'}).reset_index()
    f.columns = ['route_station', '11~12_takeoff_sum','11~12_takeoff_mean']
    f = f[['11~12_takeoff_sum','11~12_takeoff_mean']]

    g = data.groupby(['route_station'])['1820_r_mean'].agg({'sum', 'mean'}).reset_index()
    g.columns = ['route_station', '1820_r_mean_sum','1820_r_mean_mean']
    g = g[['1820_r_mean_sum','1820_r_mean_mean']]

    h = data.groupby(['route_station'])['1820_r_sum'].agg({'sum', 'mean'}).reset_index()
    h.columns = ['route_station', '1820_r_sum_sum','1820_r_sum_mean']
    h = h[['1820_r_sum_sum','1820_r_sum_mean']]

    i = data.groupby(['route_station'])['1820_rs_mean'].agg({'sum', 'mean'}).reset_index()
    i.columns = ['route_station', '1820_rs_mean_sum','1820_rs_mean_mean']
    i = i[['1820_rs_mean_sum','1820_rs_mean_mean']]

    j = data.groupby(['route_station'])['1820_rs_sum'].agg({'sum', 'mean'}).reset_index()
    j.columns = ['route_station', '1820_rs_sum_sum','1820_rs_sum_mean']
    j = j[['1820_rs_sum_sum','1820_rs_sum_mean']]

    k = data.groupby(['route_station'])['1820_s_mean'].agg({'sum', 'mean'}).reset_index()
    k.columns = ['route_station', '1820_s_mean_sum','1820_s_mean_mean']
    k = k[['1820_s_mean_sum','1820_s_mean_mean']]

    l = data.groupby(['route_station'])['1820_s_sum'].agg({'sum', 'mean'}).reset_index()
    l.columns = ['route_station', '1820_s_sum_sum','1820_s_sum_mean']
    l = l[['1820_s_sum_sum','1820_s_sum_mean']]

    m = data.groupby(['route_station'])['1820_w_mean'].agg({'sum', 'mean'}).reset_index()
    m.columns = ['route_station', '1820_w_mean_sum','1820_w_mean_mean']
    m = m[['1820_w_mean_sum','1820_w_mean_mean']]

    n = data.groupby(['route_station'])['1820_w_sum'].agg({'sum', 'mean'}).reset_index()
    n.columns = ['route_station', '1820_w_sum_sum','1820_w_sum_mean']
    n = n[['1820_w_sum_sum','1820_w_sum_mean']]

    o = data.groupby(['route_station'])['68a'].agg({'sum', 'mean'}).reset_index()
    o.columns = ['route_station', '68a_sum','68a_mean']
    o = o[['68a_sum','68a_mean']]

    p = data.groupby(['route_station'])['68b'].agg({'sum', 'mean'}).reset_index()
    p.columns = ['route_station', '68b_sum','68b_mean']
    p = p[['68b_sum','68b_mean']]

    q = data.groupby(['route_station'])['6~7_ride'].agg({'sum', 'mean'}).reset_index()
    q.columns = ['route_station', '6~7_ride_sum','6~7_ride_mean']
    q = q[['6~7_ride_sum','6~7_ride_mean']]

    r = data.groupby(['route_station'])['6~7_takeoff'].agg({'sum', 'mean'}).reset_index()
    r.columns = ['route_station', '6~7_takeoff_sum','6~7_takeoff_mean']
    r = r[['6~7_takeoff_sum','6~7_takeoff_mean']]

    s = data.groupby(['route_station'])['7~8_ride'].agg({'sum', 'mean'}).reset_index()
    s.columns = ['route_station', '7~8_ride_sum','7~8_ride_mean']
    s = s[['7~8_ride_sum','7~8_ride_mean']]

    t = data.groupby(['route_station'])['7~8_takeoff'].agg({'sum', 'mean'}).reset_index()
    t.columns = ['route_station', '7~8_takeoff_sum','7~8_takeoff_mean']
    t = t[['7~8_takeoff_sum','7~8_takeoff_mean']]

    u = data.groupby(['route_station'])['810a'].agg({'sum', 'mean'}).reset_index()
    u.columns = ['route_station', '810a_sum','810a_mean']
    u = u[['810a_sum','810a_mean']]

    v = data.groupby(['route_station'])['810b'].agg({'sum', 'mean'}).reset_index()
    v.columns = ['route_station', '810b_sum','810b_mean']
    v = v[['810b_sum','810b_mean']]

    w = data.groupby(['route_station'])['8~9_ride'].agg({'sum', 'mean'}).reset_index()
    w.columns = ['route_station', '8~9_ride_sum','8~9_ride_mean']
    w = w[['8~9_ride_sum','8~9_ride_mean']]

    x = data.groupby(['route_station'])['8~9_takeoff'].agg({'sum', 'mean'}).reset_index()
    x.columns = ['route_station', '8~9_takeoff_sum','8~9_takeoff_mean']
    x = x[['8~9_takeoff_sum','8~9_takeoff_mean']]

    y = data.groupby(['route_station'])['9~10_ride'].agg({'sum', 'mean'}).reset_index()
    y.columns = ['route_station', '9~10_ride_sum','9~10_ride_mean']
    y = y[['9~10_ride_sum','9~10_ride_mean']]

    z = data.groupby(['route_station'])['9~10_takeoff'].agg({'sum', 'mean'}).reset_index()
    z.columns = ['route_station', '9~10_takeoff_sum','9~10_takeoff_mean']
    z = z[['9~10_takeoff_sum','9~10_takeoff_mean']]
    
    df = pd.concat([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z],axis=1)
    df = pd.merge(data, df, how='left', on='route_station')
    
    return df

In [0]:
data = morning()

data.shape

In [0]:
# 배차 간격
# 배차 간격 변수를 만들 때는 많은 시간이 소요됩니다. 따라서 위의 과정의 결과를 데이터프레임으로 구성해놓았습니다.
train['bus_route_id'] = train['bus_route_id'].astype(np.int64)
test['bus_route_id'] = test['bus_route_id'].astype(np.int64)

In [0]:
bts['geton_time2'] = pd.to_datetime(bts['geton_time'])

f = bts.groupby(['geton_date','geton_time2','geton_station_code','bus_route_id'])['user_count'].\
agg([('탑승객_수','sum')]).reset_index().\
sort_values(by=['geton_date','geton_station_code','bus_route_id','geton_time2'], ascending=True).reset_index()

f['index'] = list(range(0,len(f)))

In [0]:
time = []

for i in range(0,len(f)-1):

    if ((f.iloc[i].geton_date == f.iloc[i+1].geton_date) &\
        (f.iloc[i].geton_station_code == f.iloc[i+1].geton_station_code) &\
        (f.iloc[i].bus_route_id == f.iloc[i+1].bus_route_id)):

        time.append(f.iloc[i+1].geton_time2 - f.iloc[i].geton_time2)

    else:
        time.append(0)

time.insert(0, '0')

In [0]:
def get_sec(time_str):

    h, m, s = time_str.split(':')

    return int(h) * 3600 + int(m) * 60 + int(s)

In [0]:
def bus_interval() :

    f['time'] = time
    f['time2'] = f['time'].astype(str).str[7:]


    interval = f.copy()
    interval['time2'] = interval['time2'].astype(str).replace('','00:00:00')
    interval['bus_route_id'] = interval['bus_route_id'].astype(object)

    time4 = []

    for i in interval['time2'] :

        time4.append(get_sec(i))

    interval['time4'] = time4
    interval['time4'] = (interval['time4'] / 60).astype(int)

    interval = interval[interval['time4'] &gt; 3] # 간격이 3분보다 작은 것 제외 
    interval = interval[interval['time4'] &lt; 180] # 간격이 3시간보다 큰 것 제외

    interval = interval.groupby('bus_route_id')['time4'].agg([('bus_interval', 'mean')]).reset_index()
    interval['bus_interval'] = interval['bus_interval'].astype(int)

    # 나중에 시간을 절약하기 위해 csv 파일로 저장
    interval.to_csv('bus_interval_final.csv', index = False)

    print('success.. !')

In [0]:
bus_interval()

In [0]:
# 이 과정은 상당히 오래걸리는 관계로 따로 데이터셋을 만들어서 구성해놓았습니다.
bus_interval = pd.read_csv("bus_interval_final.csv")

In [0]:
data['bus_route_id'] = data['bus_route_id'].astype(np.int64)

In [0]:
data = pd.merge(data, bus_interval, how = 'left', on = 'bus_route_id')

'bus_interval'컬럼에서 na값을 가지는 행들을 분석해본 결과, 대부분 18~20시의 탑승 인원이 거의 없는 bus_route_id 와 station_code 였다. 따라서 탑승 인원이 별로 없을 것이라고 예상되는 버스는 배차 간격이 길 것이라고 판단하여 na값을 '9999' 로 채워주었다.


In [0]:
data['bus_interval'] = data['bus_interval'].fillna(9999)

# Label encoding feature
#### bus_route_id, station_name, route_station_weekday(bus_route_id + weekday의 조합), route_station(bus_route_id + station_code 의 조합) 총 4개를 라벨인코딩 해줌

In [0]:
labelencoder = LabelEncoder()
df_encode = data[['bus_route_id','station_code', 'route_station_weekday', 'route_station']]
df_encoded = df_encode.apply(labelencoder.fit_transform)

data['bus_route_id2']=df_encoded['bus_route_id']
data['station_code2']=df_encoded['station_code']
data['route_station_weekday2']=df_encoded['route_station_weekday']
data['route_station2']=df_encoded['route_station']

data.shape

In [0]:
# weather
def weather() :

    weather_data = pd.read_csv('weather.csv', encoding = 'utf-8')
    weather_data['id'] = range(0,46)

    a = pd.DataFrame(data.date.unique(), columns=['date']) ; a['id'] = range(0,46)
    weather_data = pd.merge(a, weather_data)
    weather_data = weather_data[['date','현재일기_10','체감온도_10','일강수_10']]
    weather_data = weather_data.replace(' ', 0)
    df = pd.merge(data, weather_data, on='date')
    # label_encoder
    labelencoder = LabelEncoder()
    df_encode = df[['현재일기_10']]
    df_encoded = df_encode.apply(labelencoder.fit_transform)
    df['현재일기_10']=df_encoded['현재일기_10']
    # object-&gt;float 변수변환
    df['현재일기_10'] = df['현재일기_10'].astype(float)
    df['체감온도_10'] = df['체감온도_10'].astype(float)
    df['일강수_10'] = df['일강수_10'].astype(float)
    return df

In [0]:
data = weather()

data.shape

In [0]:
# weekday (data)
data['date'] = pd.to_datetime(data['date'])
data['weekday'] = data['date'].dt.weekday
data = pd.get_dummies(data,columns=['weekday'])
data['weekday'] = data['date'].dt.weekday
data.shape

In [0]:
# in - out
data['in_out'].value_counts()
data['in_out'] = data['in_out'].map({'시내':0,'시외':1})

data.shape

In [0]:
# 좌표데이터를 이용 변수
# 좌표데이터를 이용한 변수를 만들 때는 많은 시간이 소요됩니다.
coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
data['dis_jejusi'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejusi).km for i in range(len(data))]

coords_jejusicheong1 = (33.49892, 126.53035) #제주시청(광양방면)의 위도 경도
coords_jejuairport = (33.50661, 126.49345) #제주국제공항(구제주방면)의 위도 경도
coords_hallahosp = (33.48963, 126.486) #한라병원의 위도 경도
coords_rotary = (33.49143, 126.49678) # 제주도청신제주로터리의 위도 경도
coords_jejucenterhigh = (33.48902, 126.5392) #제주중앙여자고등학교의 위도 경도
coords_jejumarket = (33.51315, 126.52706) #동문시장의 위도 경도
coords_jejusclass = (33.47626, 126.48141) #제주고등학교/중흥S클래스의 위도 경도
coords_centerroad = (33.51073, 126.5239) #중앙로(국민은행)의 위도 경도
coords_fiveway = (33.48667, 126.48092) # 노형오거리의 위도 경도
coords_law = (33.49363, 126.53476) # 제주지방법원(광양방면)의 위도 경도

data['dis_jejusicheong1'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejusicheong1).km for i in range(len(data))]
data['dis_jejuairport'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejuairport).km for i in range(len(data))]
data['dis_hallahosp'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_hallahosp).km for i in range(len(data))]
data['dis_rotary'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_rotary).km for i in range(len(data))]
data['dis_jejucenterhigh'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejucenterhigh).km for i in range(len(data))]
data['dis_jejumarket'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejumarket).km for i in range(len(data))]
data['dis_jejusclass'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejusclass).km for i in range(len(data))]
data['dis_centerroad'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_centerroad).km for i in range(len(data))]
data['dis_fiveway'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_fiveway).km for i in range(len(data))]
data['dis_law'] = [geopy.distance.vincenty((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_law).km for i in range(len(data))]

data.shape

In [0]:
# 출근 시간의 총 승객 수
data['ride_sum'] = data['6~7_ride'] + data['7~8_ride'] + data['8~9_ride'] + data['9~10_ride'] + data['10~11_ride'] + data['11~12_ride'] 
data['takeoff_sum'] = data['6~7_takeoff'] + data['7~8_takeoff'] + data['8~9_takeoff'] + data['9~10_takeoff'] + data['10~11_takeoff'] + data['11~12_takeoff'] 

data.shape

In [0]:
# 날짜 및 시간대 별 총 승객수
f = data.groupby('date')['6~7_ride'].agg([('6~7_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['7~8_ride'].agg([('7~8_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['8~9_ride'].agg([('8~9_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['9~10_ride'].agg([('9~10_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['10~11_ride'].agg([('10~11_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

data.shape

In [0]:
# 주말, 주중
def h(x):
    if x ==5:
        return 1
    elif x==6:
        return 1
    else:
        return 0

In [0]:
data['weekend'] = data['weekday'].apply(h)

data.shape

In [0]:
# 연휴
def g(x):
    if x in ['2019-09-12','2019-09-13','2019-09-14','2019-10-03','2019-10-09']:
        return 1
    else:
        return 0

In [0]:
data['holiday'] = data['date'].apply(g) 

data.shape

In [0]:
# 요일 별 평균 승객 수
def week_mean() :

    df = data.reset_index(drop=True)
    df.groupby('weekday')['18~20_ride'].mean()
    df['weekdaymean']= 1

    index0 = df.query('weekday==0').index
    index1 = df.query('weekday==1').index
    index2 = df.query('weekday==2').index
    index3 = df.query('weekday==3').index
    index4 = df.query('weekday==4').index
    index5 = df.query('weekday==5').index
    index6 = df.query('weekday==6').index

    df.iloc[index0,-1] = 1.343710
    df.iloc[index1,-1] = 1.375319
    df.iloc[index2,-1] = 1.430856
    df.iloc[index3,-1] = 1.256710
    df.iloc[index4,-1] = 1.067439
    df.iloc[index5,-1] = 1.062123
    df.iloc[index6,-1] = 1.034282

    return df

In [0]:
data = week_mean()

data.shape

In [0]:
# 시내 및 시외버스 별 평균 탑승 승객
data['in_out_mean'] = 1
inindex = data.query('in_out == "시내"').index
outindex = data.query('in_out == "시외"').index

data.iloc[inindex,-1] = 1.228499
data.iloc[outindex,-1] = 2.044345
data['congestion'] = data['congestion'].astype('int64')

data.shape

In [0]:
# 카테고리별 승객 수
def category_people() :
    bts['bus_route_id'] = bts['bus_route_id'].astype(str)

    f = bts.groupby(['bus_route_id','user_category'])['user_count'].agg([('승객수', 'sum')]).reset_index()

    g = pd.pivot_table(f, values='승객수', index='bus_route_id', columns='user_category',fill_value=0).reset_index()
    g.columns = ['bus_route_id', 'adult','kids','teen','elder','jang','jang2','ugong','ugong2']
    g = g[['bus_route_id', 'adult','kids','teen','elder']]

    # merge
    df = pd.merge(data, g, how='left', on='bus_route_id')

    # na preprocessing -&gt; mean value
    df['adult'] = df['adult'].fillna(2363.077778)
    df['kids'] = df['kids'].fillna(60.426984)
    df['teen'] = df['teen'].fillna(448.277778)
    df['elder'] = df['elder'].fillna(751.309524)
                 
    return df

In [0]:
data['bus_route_id'] = data['bus_route_id'].astype(str)

In [0]:
data = category_people()

data.shape

In [0]:
# Category별 승객의 비율
def category_people_ratio() :
    
    a = bts.groupby('bus_route_id')['user_count'].agg([('전체', 'sum')]).reset_index()
    b = bts.groupby(['bus_route_id','user_category'])['user_count'].agg([('승객수', 'sum')]).reset_index()

    c = pd.merge(b, a, on='bus_route_id')
    c['비율'] = c['승객수']/c['전체']
    c = pd.pivot_table(c, values='비율', index='bus_route_id', columns='user_category',fill_value=0).reset_index()
    c.columns = ['bus_route_id', 'adult_prop','kids_prop','teen_prop','elder_prop','jang_prop','jang2_prop','ugong_prop','ugong2_prop']
    f = c[['bus_route_id', 'adult_prop','kids_prop','teen_prop','elder_prop']]

    df = pd.merge(data, f, how='left', on='bus_route_id')

    # na preprocessing -&gt; mean value
    df['adult_prop'] = df['adult_prop'].fillna(0.549702)
    df['kids_prop'] = df['kids_prop'].fillna(60.426984)
    df['teen_prop'] = df['teen_prop'].fillna(0.019902)
    df['elder_prop'] = df['elder_prop'].fillna(0.235848)

    return df

In [0]:
ategory_people_ratio()

data.shape

In [0]:
# 각 동,읍,면별 직업군별 비율, 평균소득액, 평균소비액
# jeju_finantial_life_data (외부데이터) 사용하여 피쳐 생성
# train, test 데이터의 'latitude', 'longitude'열, jeju_financial_life_data의 'X_AXIS', 'Y_AXIS'열을 지오코딩 프로그램을 사용하여 주소로 변환하였다. 
# 변환한 주소를 전처리 하여 동 기준으로 두 데이터를 합쳐주었다. 그 뒤 파생변수를 생성하였다.
# data_address.csv는 data의 주소가, life_address.csv는 jeju_finantial_life_data의 주소가 담겨있다.

def jeoju_love() :
    
    loc_data = pd.read_csv("data_address.csv", encoding='cp949')
    loc_life = pd.read_csv("life_address.csv", encoding='cp949')
    
    loc_data = loc_data[['location','dong', 'si']]
    loc_life = loc_life[['location','dong', 'si']]

    df = pd.merge(data, loc_data, how='left', on='location')
    
    jeju_life = pd.read_csv("jeju_financial_life_data.csv")
    jeju_life['location'] = jeju_life['x_axis'].astype(str).str[:10] + ',' + jeju_life['y_axis'].astype(str).str[:10]
    jeju_life2 = pd.merge(jeju_life, loc_life, how='left', on='location')

    dong_f1 = jeju_life2.groupby(['dong'])[['job_majorc', 'job_smallc', 'job_public', 'job_profession', 'job_self','vehicle_own_rat', 'avg_income', 'med_income', 'avg_spend']].mean().reset_index()
    dong_f1.columns=['dong','mean_job_majorc', 'mean_job_smallc', 'mean_job_public', 'mean_job_profession', 'mean_job_self','mean_vehicle_own_rat', 'mean_avg_income', 'mean_med_income', 'mean_avg_spend']

    dong_f2 = jeju_life2.groupby(['dong'])[['job_majorc', 'job_smallc', 'job_public', 'job_profession', 'job_self','vehicle_own_rat', 'avg_income', 'med_income', 'avg_spend']].sum().reset_index()
    dong_f2.columns=['dong','sum_job_majorc', 'sum_job_smallc', 'sum_job_public', 'sum_job_profession', 'sum_job_self','sum_vehicle_own_rat', 'sum_avg_income', 'sum_med_income', 'sum_avg_spend']

    dong_f3 = (jeju_life2.groupby(['dong'])['job_majorc', 'job_smallc', 'job_public', 'job_profession', 'job_self','vehicle_own_rat', 'avg_income', 'med_income', 'avg_spend'].sum()/jeju_life2.groupby(['dong'])['job_majorc', 'job_smallc', 'job_public', 'job_profession', 'job_self','vehicle_own_rat', 'avg_income', 'med_income', 'avg_spend'].sum().sum()).reset_index()
    dong_f3.columns = ['dong','rate_job_majorc', 'rate_job_smallc', 'rate_job_public', 'rate_job_profession', 'rate_job_self','rate_vehicle_own_rat', 'rate_avg_income', 'rate_med_income', 'rate_avg_spend']

    m_1 = pd.merge(dong_f1, dong_f2, how='left', on='dong')
    m_2 = pd.merge(m_1, dong_f3, how='left', on='dong')
    df = pd.merge(df, m_2, how='left', on='dong')

    #  # na preprocessing -&gt; mean value
    df['mean_job_majorc'] = df['mean_job_majorc'].fillna(0.024219)
    df['mean_job_smallc'] = df['mean_job_smallc'].fillna(0.145757)
    df['mean_job_public'] = df['mean_job_public'].fillna(0.032768)
    df['mean_job_profession'] = df['mean_job_profession'].fillna(0.014855)
    df['mean_job_self'] = df['mean_job_self'].fillna(0.222090)
    df['mean_vehicle_own_rat'] = df['mean_vehicle_own_rat'].fillna(0.041161)
    df['mean_avg_income'] = df['mean_avg_income'].fillna(34221420)
    df['mean_med_income'] = df['mean_med_income'].fillna(30645290)
    df['mean_avg_spend'] = df['mean_avg_spend'].fillna(4224923)

    df['sum_job_majorc'] = df['sum_job_majorc'].fillna(3.717861e+00)
    df['sum_job_smallc'] = df['sum_job_smallc'].fillna(2.078142e+01)
    df['sum_job_public'] = df['sum_job_public'].fillna(4.747755e+00)
    df['sum_job_profession'] = df['sum_job_profession'].fillna(2.169554e+00)
    df['sum_job_self'] = df['sum_job_self'].fillna(3.044199e+01)
    df['sum_vehicle_own_rat'] = df['sum_vehicle_own_rat'].fillna(5.609080e+00)
    df['sum_avg_income'] = df['sum_avg_income'].fillna(4.998226e+09)
    df['sum_med_income'] = df['sum_med_income'].fillna(4.455924e+09)
    df['sum_avg_spend'] = df['sum_avg_spend'].fillna(6.147678e+08)

    df['rate_job_majorc'] = df['rate_job_majorc'].fillna(1.388889e-02)
    df['rate_job_smallc'] = df['rate_job_smallc'].fillna(1.388889e-02)
    df['rate_job_public'] = df['rate_job_public'].fillna(1.388889e-02)
    df['rate_job_profession'] = df['rate_job_profession'].fillna(1.388889e-02)
    df['rate_job_self'] = df['rate_job_self'].fillna(1.388889e-02)
    df['rate_vehicle_own_rat'] = df['rate_vehicle_own_rat'].fillna(1.388889e-02)
    df['rate_avg_income'] = df['rate_avg_income'].fillna(1.388889e-02)
    df['rate_med_income'] = df['rate_med_income'].fillna(1.388889e-02)
    df['rate_avg_spend'] = df['rate_avg_spend'].fillna(1.388889e-02)

    return df

In [0]:
data = jeoju_love()

data.shape

In [0]:
# 수요가 많을 것으로 예상되는 정류장
g = data[data['station_name'].str.contains('고등학교')]
highschool = list(g['station_name'].unique())

g = data[data['station_name'].str.contains('대학교')]
university = list(g['station_name'].unique())

In [0]:
def f(x):
    if x in highschool:
        return 1
    elif x in university:
        return 1
    else:
        return 0

In [0]:
data['school'] = data['station_name'].apply(f) 

In [0]:
g = data[data['station_name'].str.contains('환승')]
transfer = list(g['station_name'].unique())

g = data[data['station_name'].str.contains('공항')]
airport = list(g['station_name'].unique())

g = data[data['station_name'].str.contains('터미널')]
terminal = list(g['station_name'].unique())

In [0]:
def f(x):
    if x in transfer:
        return 1
    elif x in airport:
        return 1
    elif x in terminal:
        return 1
    else:
        return 0

In [0]:
data['transfer'] = data['station_name'].apply(f) 

data.shape

In [0]:
# 동 - 라벨인코딩
labelencoder = LabelEncoder()
df_encode = data[['dong']]
df_encoded = df_encode.apply(labelencoder.fit_transform)

data['dong2']=df_encoded['dong']

data.shape

In [0]:
# 측정소와 정류장 사이 거리 계산
def dist() :
    jeju=(33.51411, 126.52969) # 제주 측정소 근처
    gosan=(33.29382, 126.16283) #고산 측정소 근처
    seongsan=(33.38677, 126.8802) #성산 측정소 근처
    po=(33.24616, 126.5653) #서귀포 측정소 근처

    t1 = [geopy.distance.vincenty( (i,j), jeju).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
    t2 = [geopy.distance.vincenty( (i,j), gosan).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
    t3 = [geopy.distance.vincenty( (i,j), seongsan).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
    t4 = [geopy.distance.vincenty( (i,j), po).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]

    data['dis_jeju'] = t1
    data['dis_gosan']=t2
    data['dis_seongsan']=t3
    data['dis_po']=t4

    total = pd.DataFrame(list(zip( t1,t2,t3,t4)),columns=['jeju','gosan','seongsan','po'] )
    data['dist_name'] = total.apply(lambda x: x.argmin(), axis=1)
    
    return data

In [0]:
data = dist()

data.shape

In [0]:
# 날씨 관련 변수
# 중급자코드를 참고하면 rain3.csv를 만들 수 있다.
rain3 = pd.read_csv("rain3.csv")

# train, test의 변수명과 통일시키고, NaN의 값은 0.0000으로 변경
rain3 = rain3.rename(columns={"일시":"date","지점":"dist_name"})
rain3 = rain3.fillna(0.00000)
rain3['date'] = pd.to_datetime(rain3['date'])

rain3.shape

In [0]:
data = pd.merge(data, rain3, how='left',on=['dist_name','date'])
data = pd.get_dummies(data,columns=['dist_name'])
data = pd.get_dummies(data,columns=['si'])

data.shape

In [0]:
# rainy_day
# 비 오는날=1, 비 안오는 날=0
def f(x):
    if x == 0:
        return 0
    else:
        return 1

In [0]:
data['rainy_day'] = data['강수량(mm)'].apply(f)

data.shape

In [0]:
# 승 하차 시간대 통합 변수 (t ~ t+3)
# t~t+1, t+1~t+2, t+2~t+3 시간대 승하차인원을 합하여 t~t+3 시간대 승하차인원 변수를 만듬
data['69a'] = data['6~7_ride']+data['7~8_ride']+data['8~9_ride']
data['912a']=data['9~10_ride']+data['10~11_ride']+data['11~12_ride']

data['69b'] = data['6~7_takeoff']+data['7~8_takeoff']+data['8~9_takeoff']
data['912b'] = data['9~10_takeoff']+data['10~11_takeoff']+data['11~12_takeoff']

data.shape

In [0]:
# MAKE DATASET
# train, test 데이터를 만들어준다.
train_data = data.query('cue=="0"').reset_index()
test_data = data.query('cue=="1"').reset_index()

train_data.shape, test_data.shape

In [0]:
# 요약통계량을 통한 변수를 만드는 과정에서 NA가 발생한다.

# target variable을 분리한 후 데이터를 저장한다.
y_train = train_data[['18~20_ride']]

In [0]:
train_data.shape, test_data.shape, y_train.shape

In [0]:
train_data.to_csv('X_train.csv', index = False, encoding = 'utf-8')
test_data.to_csv('X_test.csv', index = False, encoding = 'utf-8')
y_train.to_csv('y_train.csv', index = False, encoding = 'utf-8')

print('save ..!')

In [0]:
train_data = pd.read_csv('X_train.csv', encoding = 'utf-8')
test_data = pd.read_csv('X_test.csv', encoding = 'utf-8')
y_train = pd.read_csv('y_train.csv', encoding = 'utf-8')

In [0]:
# COLUMNS
input_var_0=['in_out','latitude','longitude','6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride', '11~12_ride',
           '6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
           'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 
           'dis_jejusi', 'dis_jejusicheong1','dis_jejuairport','dis_hallahosp', 'dis_rotary','dis_jejucenterhigh',
           'dis_jejumarket', 'dis_centerroad', 'dis_jejusclass', 'dis_fiveway', 'dis_law',
           'weekend', 'holiday', 'ride_sum', 'takeoff_sum', '1820_rs_mean', '1820_r_mean', '1820_s_mean', 'congestion',
           'station_code2', 'bus_route_id2', '일강수_10', '현재일기_10', '체감온도_10',
           '6~7_all_ride_number', '7~8_all_ride_number', '8~9_all_ride_number', '9~10_all_ride_number', '10~11_all_ride_number',
           '1820_w_mean','in_out_mean','weekdaymean','adult','kids','teen','elder','adult_prop', 'kids_prop', 'teen_prop', 'elder_prop',
           'mean_job_majorc', 'mean_job_smallc', 'mean_job_public', 'mean_job_profession', 'mean_job_self',
           'mean_vehicle_own_rat', 'mean_avg_income', 'mean_med_income', 'mean_avg_spend', 
           'rate_job_majorc', 'rate_job_smallc', 'rate_job_public', 'rate_job_profession', 'rate_job_self', 
           'rate_vehicle_own_rat', 'rate_avg_income', 'rate_med_income','rate_avg_spend',
           'sum_job_majorc', 'sum_job_smallc', 'sum_job_public', 'sum_job_profession', 'sum_job_self', 
           'sum_vehicle_own_rat', 'sum_avg_income', 'sum_med_income','sum_avg_spend',
           '68a', '810a', '1012a', '68b', '810b', '1012b','69a','912a','69b','912b',
           'dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po', '기온(°C)', '강수량(mm)',
           'dist_name_gosan', 'dist_name_jeju', 'dist_name_po', 'dist_name_seongsan', 'si_서귀포시', 'si_제주시',
           'school', 'transfer', 'dong2', 'rainy_day']

In [0]:
input_var_1=['in_out','latitude','longitude','6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride', '11~12_ride',
           '6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
           'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 
           'dis_jejusi', 'dis_jejusicheong1','dis_jejuairport','dis_hallahosp', 'dis_rotary','dis_jejucenterhigh',
           'dis_jejumarket', 'dis_centerroad', 'dis_jejusclass', 'dis_fiveway', 'dis_law',
           'weekend', 'holiday', 'ride_sum', 'takeoff_sum', '1820_rs_mean', '1820_r_mean', '1820_s_mean', 'congestion',
           'station_code2', 'bus_route_id2', '일강수_10', '현재일기_10', '체감온도_10',
           '6~7_all_ride_number', '7~8_all_ride_number', '8~9_all_ride_number', '9~10_all_ride_number', '10~11_all_ride_number',
           '1820_w_mean','in_out_mean','weekdaymean','adult','kids','teen','elder','adult_prop', 'kids_prop', 'teen_prop', 'elder_prop',
           'mean_job_majorc', 'mean_job_smallc', 'mean_job_public', 'mean_job_profession', 'mean_job_self',
           'mean_vehicle_own_rat', 'mean_avg_income', 'mean_med_income', 'mean_avg_spend', 
           'rate_job_majorc', 'rate_job_smallc', 'rate_job_public', 'rate_job_profession', 'rate_job_self', 
           'rate_vehicle_own_rat', 'rate_avg_income', 'rate_med_income','rate_avg_spend',
           'sum_job_majorc', 'sum_job_smallc', 'sum_job_public', 'sum_job_profession', 'sum_job_self', 
           'sum_vehicle_own_rat', 'sum_avg_income', 'sum_med_income','sum_avg_spend',
           '68a', '810a', '1012a', '68b', '810b', '1012b','69a','912a','69b','912b',
           'dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po', '기온(°C)', '강수량(mm)',
           'dist_name_gosan', 'dist_name_jeju', 'dist_name_po', 'dist_name_seongsan', 'si_서귀포시', 'si_제주시',
           'school', 'transfer', 'dong2', 'rainy_day']

In [0]:
input_var_2=['in_out','latitude','longitude','6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride', '11~12_ride',
           '6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
           'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 
           'dis_jejusi', 'dis_jejusicheong1','dis_jejuairport','dis_hallahosp', 'dis_rotary','dis_jejucenterhigh',
           'dis_jejumarket', 'dis_centerroad', 'dis_jejusclass', 'dis_fiveway', 'dis_law',
           'weekend', 'holiday', 'ride_sum', 'takeoff_sum', '1820_rs_mean', '1820_r_mean', '1820_s_mean', 'congestion',
           'station_code2', 'bus_route_id2', '일강수_10', '현재일기_10', '체감온도_10',
           '6~7_all_ride_number', '7~8_all_ride_number', '8~9_all_ride_number', '9~10_all_ride_number', '10~11_all_ride_number',
           '1820_w_mean','in_out_mean','weekdaymean','adult','kids','teen','elder','adult_prop', 'kids_prop', 'teen_prop', 'elder_prop',
           'mean_job_majorc', 'mean_job_smallc', 'mean_job_public', 'mean_job_profession', 'mean_job_self',
           'mean_vehicle_own_rat', 'mean_avg_income', 'mean_med_income', 'mean_avg_spend', 
           'rate_job_majorc', 'rate_job_smallc', 'rate_job_public', 'rate_job_profession', 'rate_job_self', 
           'rate_vehicle_own_rat', 'rate_avg_income', 'rate_med_income','rate_avg_spend',
           'sum_job_majorc', 'sum_job_smallc', 'sum_job_public', 'sum_job_profession', 'sum_job_self', 
           'sum_vehicle_own_rat', 'sum_avg_income', 'sum_med_income','sum_avg_spend',
           '68a', '810a', '1012a', '68b', '810b', '1012b','69a','912a','69b','912b',
           'dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po', '기온(°C)', '강수량(mm)',
           'dist_name_gosan', 'dist_name_jeju', 'dist_name_po', 'dist_name_seongsan', 'si_서귀포시', 'si_제주시',
           'school', 'transfer', 'dong2', 'rainy_day']

In [0]:
input_var_3 =['in_out','latitude','longitude','6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride', '11~12_ride',
           '6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
           'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 
           'dis_jejusi', 'dis_jejusicheong1','dis_jejuairport','dis_hallahosp', 'dis_rotary','dis_jejucenterhigh',
           'dis_jejumarket', 'dis_centerroad', 'dis_jejusclass', 'dis_fiveway', 'dis_law',
           'weekend', 'holiday', 'ride_sum', 'takeoff_sum', '1820_rs_mean', '1820_r_mean', '1820_s_mean', 'congestion',
           'station_code2', 'bus_route_id2', '일강수_10', '현재일기_10', '체감온도_10',
           '6~7_all_ride_number', '7~8_all_ride_number', '8~9_all_ride_number', '9~10_all_ride_number', '10~11_all_ride_number',
           '1820_w_mean','in_out_mean','weekdaymean','adult','kids','teen','elder','adult_prop', 'kids_prop', 'teen_prop', 
           'elder_prop', 'mean_job_majorc', 'mean_job_smallc',
           'mean_job_public', 'mean_job_profession', 'mean_job_self','mean_vehicle_own_rat',
          '68a', '810a', '1012a', '68b', '810b', '1012b',
          'dis_jeju', 'dis_gosan','dis_seongsan', 'dis_po','기온(°C)', '강수량(mm)', 
           'dist_name_gosan', 'dist_name_jeju','dist_name_po', 'dist_name_seongsan']

In [0]:
input_var_4 = ['sum_avg_spend', '68a', '810a', 'si_제주시',  'dong2' , 'bus_interval', 'dis_jejuairport', 'ride_sum',
             'takeoff_sum', '1820_rs_mean', '1820_rs_sum','1820_r_mean','1820_r_sum', '1820_s_mean',
             '1820_s_sum','congestion', 'bus_route_id2', '6~7_all_ride_number', '7~8_all_ride_number',
             '8~9_all_ride_number', '1012a_mean', '1012b_sum','10~11_ride_sum', '10~11_takeoff_sum', '11~12_ride_sum',
             '11~12_takeoff_sum', '1820_r_mean_sum', '1820_r_mean_mean',
             '1820_r_sum_sum', '1820_r_sum_mean', '1820_rs_mean_sum',
             '1820_s_mean_sum', '1820_s_mean_mean', '1820_s_sum_sum',
             '1820_s_sum_mean', '1820_w_mean_sum', '1820_w_mean_mean',
             '1820_w_sum_mean', '68a_sum', '68a_mean', '68b_sum', 'in_out', 'latitude', 'longitude',
             '6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff',
             '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
             'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
             'weekday_5', 'weekday_6', 'dis_jejusi', '68b_mean', '6~7_ride_sum',
             '6~7_ride_mean', '6~7_takeoff_sum', '6~7_takeoff_mean', '7~8_ride_sum',
             '7~8_ride_mean', '7~8_takeoff_sum', '7~8_takeoff_mean', '810a_sum',
             '810b_sum', '8~9_ride_sum', '8~9_takeoff_sum', '8~9_takeoff_mean',
             '9~10_ride_sum', '9~10_takeoff_sum', 'route_station_weekday2']

In [0]:
# 모델 학습 및 검증
# TASK - 0
lgbm = lgb.LGBMRegressor(num_iterations = 1000, 
                                learning_rate = 0.05,
                                boosting = 'dart',
                         Metric = 'regression_l2', n_jobs=-1)

X = train_data[input_var_0]
y = y_train

In [0]:
X.columns=range(len(X.columns))

lgbm.fit(X, y)

y_pred = lgbm.predict(X)

np.sqrt(mean_squared_error(y_train, y_pred))

y_pred = lgbm.predict(test_data[input_var_0])

In [0]:
sub = pd.read_csv('submission_sample.csv')
sub['18~20_ride'] = y_pred
sub.to_csv('lgbm0=229.csv', index = False)

In [0]:
# TASK - 1
rf = RandomForestRegressor(max_features=5,
                           min_samples_leaf=1,
                           min_samples_split=2,
                           n_estimators=300,
                           random_state=1217)

X = train_data[input_var_1]
y = y_train

In [0]:
rf.fit(X, y)

y_pred = rf.predict(train_data[input_var_1])

np.sqrt(mean_squared_error(y_train, y_pred))

y_pred = rf.predict(test_data[input_var_1])

In [0]:
sub = pd.read_csv('submission_sample.csv')
sub['18~20_ride'] = y_pred
sub.to_csv('rf1=234.csv', index = False)

In [0]:
# TASK - 2
rf = RandomForestRegressor(random_state=1217,
                           max_features= 3,
                           min_samples_leaf= 2,
                           min_samples_split=2,
                           n_estimators=500)

X = train_data[input_var_2]
y = y_train

In [0]:
rf.fit(X, y)

y_pred = rf.predict(train_data[input_var_2])

np.sqrt(mean_squared_error(y_train, y_pred))

y_pred = rf.predict(test_data[input_var_2])

In [0]:
sub = pd.read_csv('submission_sample.csv')
sub['18~20_ride'] = y_pred
sub.to_csv('rf2=238.csv', index = False)

In [0]:
# TASK - 3
rf = RandomForestRegressor(max_features=3,
                           min_samples_leaf=2,
                           min_samples_split=2,
                           n_estimators=500,
                           random_state=1217)

X = train_data[input_var_3]
y = y_train

In [0]:
rf.fit(X, y)

y_pred = rf.predict(train_data[input_var_3])

np.sqrt(mean_squared_error(y_train, y_pred))

y_pred = rf.predict(test_data[input_var_3])

In [0]:
sub = pd.read_csv('submission_sample.csv')
sub['18~20_ride'] = y_pred
sub.to_csv('rf3=236.csv', index = False)

In [0]:
# TASK - 4
rf = RandomForestRegressor(max_features=3,
                           min_samples_leaf=2,
                           min_samples_split=2,
                           n_estimators=500,
                           random_state=1217)

X = train_data[input_var_4]
y = y_train

In [0]:
rf.fit(X, y)

y_pred = rf.predict(train_data[input_var_4])

np.sqrt(mean_squared_error(y_train, y_pred))

y_pred = rf.predict(test_data[input_var_4])

In [0]:
sub = pd.read_csv('submission_sample.csv')
sub['18~20_ride'] = y_pred
sub.to_csv('rf4=231.csv', index = False)

In [0]:
# ENSEMBLE
# 변수별 성능 및 상관관계 확인
items = []

file_list = ['lgbm0=229.csv', 'rf1=234.csv', 'rf2=238.csv', 'rf3=236.csv', 'rf4=231.csv']

for item in file_list :
    if item.find('.csv') is not -1 :        
        items.append(item)

print(items)

In [0]:
for i in range(0, len(items)) :
    
    item = items[i]
    df = pd.read_csv(item, engine = 'python').iloc[:,1:]
    df.columns = [items[i]]
    
    if i == 0 :
        corr_df = pd.DataFrame()
        
    corr_df = pd.concat([corr_df, df], axis = 1)

    
corr = np.array(corr_df.corr().mean(axis = 0))

In [0]:
corr_df.corr()

In [0]:
rmse = []

for i in range(0, len(items)) :
    score = items[i].split('=')[-1][:6]
    score = score.split('.')[0]
    score = float(score)
    rmse.append(score)
    
df = pd.DataFrame({'rnk': list(range(0, len(rmse))), 'rmse': rmse, 'cor': corr})
df.index = items

In [0]:
df.sort_values('cor', ascending = False)

In [0]:
plt.figure(figsize=(10,5))
g = sns.scatterplot(x="cor", y="rmse", data=df, s=30)

for line in range(0, df.shape[0]):
     g.text(df.cor[line]+0.00005 , df.rmse[line]-0.00003, 
            df.rnk[line], horizontalalignment='left', 
            size='medium', color='black', weight='semibold')
        
plt.xlim((df.cor.min()-0.00002,df.cor.max()+0.0002))
plt.ylim((df.rmse.min()-0.0002,df.rmse.max()+0.0002))

plt.grid()
plt.show()

In [0]:
# 멱평균

# 앙상블은 별도의 jupyter 파일에서 진행합니다.

# 첫 번 째 조합
# Ensemble_1 폴더를 만든 후 lgbm0=229.csv, rf4=231.csv를 위치시킨다.
# 두 파일을 멱평균을 실시한다.
# p값은 21.2로 한다.
os.mkdir('Ensemble_1')

In [0]:
# 경로를 위치시킨다.
source_files = 'C:\\Users\\yena1\\Desktop\\3rd_Solution\\'
destination_folder = 'C:\\Users\\yena1\\Desktop\\3rd_Solution\\Ensemble_1\\'

In [0]:
shutil.move(source_files + 'lgbm0=229.csv', destination_folder + 'lgbm0=229.csv')
shutil.move(source_files + 'rf4=231.csv', destination_folder + 'rf4=231.csv')

In [0]:
# 두 번 째 조합
# Ensemble_2 폴더를 만든 후 rf1=234.csv, rf3=236.csv를 위치시킨다.
# 두 파일을 멱평균을 실시한다.
# p값은 21.2로 한다.
os.mkdir('Ensemble_2')

In [0]:
source_files = 'C:\\Users\\yena1\\Desktop\\3rd_solution\\'
destination_folder = 'C:\\Users\\yena1\\Desktop\\3rd_solution\\Ensemble_2\\'

In [0]:
shutil.move(source_files + 'rf1=234.csv', destination_folder + 'rf1=234.csv')
shutil.move(source_files + 'rf3=236.csv', destination_folder + 'rf3=236.csv')

In [0]:
# 앙상블
# 멱평균
# 첫번째 조합

# 사용법: 
# 1) 스크립트를 실행하기 전에 Ensemble 폴더를 먼저 만듭니다. 
# 2) 앙상블할 submission 화일을 Ensemble 폴더에 저장합니다.
# 3) 실행하면 현재 폴더에 앙상블한 submission 화일이 생성됩니다.

# 주) 이 스크립트는 Kaggle Kernel에서 실행할 수 없고 여러분의 Jupyter Notebook에서 실행해야 합니다.

import pandas as pd
import numpy as np
import os

folder = 'Ensemble_1'

nf = 0
for f in os.listdir(folder):
    ext = os.path.splitext(f)[-1]
    if ext == '.csv': 
        s = pd.read_csv(folder+"/"+f)
    else: 
        continue
    if len(s.columns) !=2:
        continue
    if nf == 0: 
        slist = s
    else: 
        slist = pd.merge(slist, s, on="id")
    nf += 1

# 이 파라미터는 멱평균 앙상블에 있어 중요한 수치임. 최적의 수치를 찾기 바랍니다.    
p = 21

if nf &gt;= 2:
    pred = 0
    for j in range(nf): pred = pred + slist.iloc[:,j+1]**p 
    pred = pred / nf    
    pred = pred**(1/p)

    submit = pd.DataFrame({'id': slist.id, '18~20_ride': pred})
    t = pd.Timestamp.now()
    fname = "0+4.csv"
    submit.to_csv(fname, index=False)
    
    print(fname)

In [0]:
# 두번째 조합
# 사용법: 
# 1) 스크립트를 실행하기 전에 Ensemble 폴더를 먼저 만듭니다. 
# 2) 앙상블할 submission 화일을 Ensemble 폴더에 저장합니다.
# 3) 실행하면 현재 폴더에 앙상블한 submission 화일이 생성됩니다.

# 주) 이 스크립트는 Kaggle Kernel에서 실행할 수 없고 여러분의 Jupyter Notebook에서 실행해야 합니다.

import pandas as pd
import numpy as np
import os

folder = 'Ensemble_2'

nf = 0
for f in os.listdir(folder):
    ext = os.path.splitext(f)[-1]
    if ext == '.csv': 
        s = pd.read_csv(folder+"/"+f)
    else: 
        continue
    if len(s.columns) !=2:
        continue
    if nf == 0: 
        slist = s
    else: 
        slist = pd.merge(slist, s, on="id")
    nf += 1

# 이 파라미터는 멱평균 앙상블에 있어 중요한 수치임. 최적의 수치를 찾기 바랍니다.    
p = 21

if nf &gt;= 2:
    pred = 0
    for j in range(nf): pred = pred + slist.iloc[:,j+1]**p 
    pred = pred / nf    
    pred = pred**(1/p)

    submit = pd.DataFrame({'id': slist.id, '18~20_ride': pred})
    t = pd.Timestamp.now()
    fname = "1+3.csv"
    submit.to_csv(fname, index=False)
    
    print(fname)

In [0]:
# 산술 평균(가중치)
first = pd.read_csv('0+4.csv')
second = pd.read_csv('1+3.csv')
third = pd.read_csv('rf2=238.csv')

In [0]:
target = '18~20_ride'

w1, w2, w3 = 0.22, 0.30, 0.48

In [0]:
W = w1*first[target] + w2*second[target] + w3*third[target]

In [0]:
# 최종 서브미션 파일 만들기
sub = pd.read_csv('submission_sample.csv')
sub[target] = W
sub.to_csv('Final_submission.csv')

print('finish .. !')